In [24]:
import os
import re
import pandas as pd
folder = r"Z:\Laboral\World Bank\Paper - Child mortality and Climate Shocks\Outputs"
files = os.listdir(folder)
files = [f for f in files if ".tex" in f]
files

['version 1  months.tex',
 'version 1 _12 months.tex',
 'version 1 _3 months.tex',
 'version 1 _6 months.tex',
 'version 2  months.tex',
 'version 2 _12 months.tex',
 'version 2 _3 months.tex',
 'version 2 _6 months.tex']

In [25]:
def count_significative_vars(s):
    count_p01  = s.str.contains("*", regex=False).sum()
    count_p005 = s.str.contains("**", regex=False).sum()
    count_p001 = s.str.contains("***", regex=False).sum()
    return count_p01, count_p005, count_p001 

def dict_to_df(dict):
    results_df = pd.DataFrame().from_dict(dict, orient="index", columns=["p=0.1", "p=0.05", "p=0.01"]).reset_index(names="Specification")
    return results_df

In [ ]:
files

In [28]:
import re

for i, file in enumerate(files):

    filename = rf"{folder}\{file}"
    df = pd.read_csv(filename,
                    sep='&',
                    header=0,
                    skiprows=4,
                    skipfooter=3,
                    engine='python')

    keywords = "drought|excessiverain|prec" # Regex allowed

    df = df.set_index(df.columns[0]) # Set variable names in index
    df = df[df.index != ' '] # Remove standard errors
    df = df[df.index.str.contains(keywords)]

    results = {}
    for specification in df.columns:
        counts = count_significative_vars(df[specification])
        specification = re.sub(r'[^0-9a-zA-Z]+', '', specification)
        results[specification] = counts

    results_df = dict_to_df(results)    
    results_df["path"] = filename
    results_df["file"] = file
    results_df = results_df[['file', 'path', 'Specification', 'p=0.1', 'p=0.05', 'p=0.01']]

    if i==0:
        results_df.to_csv("regs_significance_all_models.csv", index=False)    
    else:
        results_df.to_csv("regs_significance_all_models.csv", mode="a", index=False, header=False)

In [29]:

df


,(1),(2),(3),(4),(5),(6),(7),(8),(9),(10),(11),(12),(13),(14),(15),(16) \\
,,,,,,,,,,,,,,,,
prec\_6\_q1,-0.015,-0.017,-0.030,-0.051,0.094,0.092,0.090,0.073,-0.128*,-0.122*,-0.123*,-0.143**,0.018,0.004,-0.001,-0.051 \\
prec\_6\_q1\_sq,0.067,0.073,0.065,0.070,-0.012,-0.012,-0.021,-0.018,0.000,-0.007,-0.008,-0.013,0.101,0.102,0.102,0.110 \\
prec\_6\_q2,0.060,0.092,0.071,0.086,-0.048,-0.041,-0.032,-0.031,0.169*,0.166*,0.176*,0.189**,-0.014,-0.001,0.007,0.015 \\
prec\_6\_q2\_sq,-0.003,-0.004,0.007,0.002,0.010,0.016,0.014,0.017,-0.049,-0.037,-0.046,-0.047,-0.180**,-0.184**,-0.198**,-0.194** \\
prec\_6\_q3,-0.057,-0.071,-0.063,-0.088,-0.011,-0.018,-0.025,-0.031,-0.274**,-0.271**,-0.274**,-0.281**,0.163,0.156,0.162,0.141 \\
prec\_6\_q3\_sq,0.123,0.126,0.128,0.156,-0.045,-0.050,-0.048,-0.043,0.008,0.004,0.007,0.005,-0.080,-0.088,-0.084,-0.081 \\
prec\_6\_30d,0.100,0.083,0.057,0.065,0.054,0.056,0.061,0.068,0.057,0.065,0.067,0.063,0.052,0.047,0.057,0.051 \\
prec\_6\_30d\_sq,,,,,0.021,0.023,0.015,0.012,-0.040,-0.042,-0.044,-0.045,0.008,0.013,0.007,0.012 \\
prec\_6\_30d3m,,,,,-0.040,-0.031,-0.031,-0.044,0.077,0.064,0.071,0.080,-0.224,-0.233*,-0.247*,-0.244 \\


In [ ]:
files = os.listdir(r"Z:\Laboral\World Bank\Paper - Child mortality and Climate Shocks\Outputs\Resultados")
files = [f for f in files if "regression_outs" in f and ".xml" in f]
files

In [ ]:
import os
from openpyxl import load_workbook
from openpyxl import Workbook
import xml.etree.ElementTree as ET

folder = r"Z:\Laboral\World Bank\Paper - Child mortality and Climate Shocks\Outputs\Resultados"
output_excel = rf'{folder}\all_regressions.xlsx'

#List all excel files in folder
excel_files = os.listdir(folder)
excel_files = [fr"{folder}\\{f}" for f in excel_files if "regression_outs" in f and ".xml" in f]
print(excel_files)

# Function to compile multiple Excel files into one
def compile_excel_files(files, output_file):
    # Create a new workbook to store all the sheets
    compiled_workbook = Workbook()
    
    # Iterate through all files in the input folder
    for filename in files:
        if filename.endswith(".xlsx"):  # Make sure it's an Excel file
           
            # Parse the XML file
            tree = ET.parse(filename)
            root = tree.getroot()
            
            # Create a new sheet for each XML file
            sheet_name = os.path.splitext(filename)[0]  # Use filename as sheet name
            compiled_sheet = compiled_workbook.create_sheet(title=sheet_name)
            
            # Extract data from XML and populate the Excel sheet
            for row_num, child in enumerate(root.iter()):
                # Assuming each element in XML corresponds to a row in Excel
                if child.tag != root.tag:  # Exclude the root element
                    compiled_sheet.append([child.tag, child.text])  # Add data to sheet
    
    
    # Save the compiled workbook to the output file
    compiled_workbook.save(output_file)

# Example usage:
compile_excel_files(files, output_excel)


In [8]:
df

,(1),(2),(3),(4),(5),(6),(7),(8),(9),(10),(11),(12),(13),(14),(15),(16) \\
,,,,,,,,,,,,,,,,
\midrule,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
prec\_q1,-0.032,0.008,0.005,-0.008,0.047,0.052,0.055,0.035,-0.054,-0.054,-0.047,-0.056,-0.127,-0.120,-0.112,-0.146 \\
,(0.155),(0.157),(0.167),(0.194),(0.066),(0.063),(0.067),(0.064),(0.068),(0.067),(0.070),(0.064),(0.100),(0.102),(0.105),(0.114) \\
prec\_q2,-0.010,-0.022,-0.041,-0.045,-0.043,-0.041,-0.046,-0.047,-0.032,-0.030,-0.027,-0.026,-0.003,-0.028,-0.032,-0.049 \\
,(0.153),(0.153),(0.144),(0.144),(0.065),(0.065),(0.065),(0.066),(0.068),(0.066),(0.068),(0.068),(0.098),(0.096),(0.105),(0.110) \\
prec\_q3,0.100,0.069,0.067,0.059,0.053,0.059,0.059,0.045,-0.172*,-0.174*,-0.175*,-0.182*,0.008,0.001,0.005,-0.005 \\
,(0.153),(0.150),(0.152),(0.160),(0.067),(0.071),(0.073),(0.074),(0.069),(0.072),(0.072),(0.079),(0.098),(0.096),(0.094),(0.104) \\
prec\_30d,0.081,0.082,0.065,0.078,0.031,0.037,0.042,0.037,0.094*,0.094*,0.102*,0.099*,-0.042,-0.051,-0.051,-0.061 \\
,(0.096),(0.098),(0.086),(0.084),(0.041),(0.042),(0.043),(0.041),(0.040),(0.040),(0.041),(0.042),(0.059),(0.057),(0.056),(0.054) \\


In [ ]:
import pandas as pd
import os
folder = r"Z:\Laboral\World Bank\Paper - Child mortality and Climate Shocks\Outputs\Resultados"
output_excel = rf'{folder}\all_regressions.xlsx'

#List all excel files in folder
files = os.listdir(folder)
files = [f for f in files if "regression_outs" in f and ".xml" in f]
print(files)

with pd.ExcelWriter(output_excel) as writer:
    for excel in files: #For each excel
        sheet_name = excel.split(".")[0].replace("regression_outs_", "") #Find the sheet name
        df = pd.read_excel(fr"{folder}/{excel}") #Create a dataframe
        df.to_excel(writer, sheet_name=sheet_name, index=False) #Write it to a sheet in the output excel
